In [18]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split,cross_val_score

In [19]:
X,y = make_classification(n_samples=10000,n_features=10,n_informative=3)

In [20]:
X.shape

(10000, 10)

In [21]:
y.shape

(10000,)

In [22]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [23]:
X_train.shape

(8000, 10)

In [24]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)

print("Decision Tree Accuracy:",accuracy_score(y_test,y_pred))

Decision Tree Accuracy: 0.9095


In [26]:
from sklearn.model_selection import cross_val_score

np.mean(cross_val_score(dt,X,y,scoring='accuracy',cv=10))

0.9186

## Bagging Classifier

In [27]:
from sklearn.ensemble import BaggingClassifier

bag = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.5,
    bootstrap=True,
    random_state=42

)

In [28]:
bag.fit(X_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.5,
                  n_estimators=500, random_state=42)

In [29]:
y_pred = bag.predict(X_test)

In [30]:
accuracy_score(y_test,y_pred)

0.944

In [33]:
bag.estimators_samples_[0].shape

(4000,)

In [34]:
bag.estimators_features_[0].shape

(10,)

## Bagging using SVM

In [36]:
from sklearn.svm import SVC
bag = BaggingClassifier(
    base_estimator=SVC(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    random_state=42
)

In [37]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Bagging using SVM",accuracy_score(y_test,y_pred))

Bagging using SVM 0.9125


## Using Pasting

In [38]:
bag = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=False,
    random_state=42,
    verbose = 1,
    n_jobs=-1
)

In [39]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Pasting classifier",accuracy_score(y_test,y_pred))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    7.4s remaining:    7.4s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    7.6s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.3s remaining:    0.3s


Pasting classifier 0.943


[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.5s finished


## Using Random Subspaces

In [40]:
bag = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=1.0,
    bootstrap=False,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)    

In [41]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Random Subspaces classifier",accuracy_score(y_test,y_pred))

Random Subspaces classifier 0.9415


In [42]:
bag.estimators_samples_[0].shape

(8000,)

In [43]:
bag.estimators_features_[0].shape

(5,)

## Random Patches

In [44]:
bag = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)

In [45]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Random Patches classifier",accuracy_score(y_test,y_pred))

Random Patches classifier 0.939


## OOB Score

In [46]:
bag = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    oob_score=True,
    random_state=42
)

In [47]:

bag.fit(X_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.25,
                  n_estimators=500, oob_score=True, random_state=42)

In [48]:
bag.oob_score_

0.948

In [49]:
y_pred = bag.predict(X_test)
print("Accuracy",accuracy_score(y_test,y_pred))
     

Accuracy 0.9425


## Bagging Tips
Bagging generally gives better results than Pasting.  
Good results come around the 25% to 50% row sampling mark  
Random patches and subspaces should be used while dealing with high dimensional data  
To find the correct hyperparameter values we can do GridSearchCV/RandomSearchCV  

## Applying GridSearchCV

In [50]:
from sklearn.model_selection import GridSearchCV

In [51]:
parameters = {
    'n_estimators': [50,100,500], 
    'max_samples': [0.1,0.4,0.7,1.0],
    'bootstrap' : [True,False],
    'max_features' : [0.1,0.4,0.7,1.0]
    }

In [52]:
search = GridSearchCV(BaggingClassifier(), parameters, cv=5)

In [ ]:
search.fit(X_train,y_train)

In [ ]:
search.best_params_


In [ ]:
search.best_score_

In [ ]:
search.best_params_
